# Some analysis of the branching fraction data in ICRP 107
This notebook checks the consistency of the branching fraction data in the <a href="http://www.icrp.org/publication.asp?id=ICRP%20Publication%20107">ICRP 107</a> decay dataset. The branching fractions for the progeny of one radionuclide should be sum to unity for physical reasons. However in practice the sum totals may be slightly different from unity as the branching fractions are only quoted to a maximum of 5 significant figures of precision in the ICRP 107 dataset.

This notebook calculates the sum of progeny branching fractions for each radionuclide in ICRP 107 and analyzes the results.

First read in a CSV file containing the ICRP 107 decay data into a DataFrame.

In [1]:
import pandas as pd
import numpy as np

icrp = pd.read_csv('icrp.csv', index_col='Radionuclide')
icrp.head()

,Element,Z,A,Metastable_state,Half-life_s,Num_decay_modes,Mode_1,Fraction_1,Progeny_1,Mode_2,Fraction_2,Progeny_2,Mode_3,Fraction_3,Progeny_3,Mode_4,Fraction_4,Progeny_4
Radionuclide,,,,,,,,,,,,,,,,,,
Ac-223,Ac,89,223,NaN,1.260000e+02,1,Alpha,0.9900,Fr-219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ac-224,Ac,89,224,NaN,1.000800e+04,2,EC (no Beta+),0.9090,Ra-224,Alpha,0.0910,Fr-220,NaN,NaN,NaN,NaN,NaN,NaN
Ac-225,Ac,89,225,NaN,8.640000e+05,1,Alpha,1.0000,Fr-221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ac-226,Ac,89,226,NaN,1.057320e+05,3,Beta-,0.8300,Th-226,EC (no Beta+),0.1700,Ra-226,Alpha,0.00006,Fr-222,NaN,NaN,NaN
Ac-227,Ac,89,227,NaN,6.870574e+08,2,Beta-,0.9862,Th-227,Alpha,0.0138,Fr-223,NaN,NaN,NaN,NaN,NaN,NaN


Find all the radionuclides where the sum of the progeny branching fractions does not equal one and add them to a DataFrame.

In [2]:
nonunity = []
for rn in icrp.index.to_list():
    sum = 0.0
    i = 1
    while True:
        if np.isnan(icrp.loc[rn, 'Fraction_'+str(i)]):
            break
        sum += icrp.loc[rn, 'Fraction_'+str(i)]
        i += 1
        if i > 4: break
    if sum != 1.0:
        nonunity.append([rn, sum])
nonunity = pd.DataFrame(nonunity, columns=['Radionuclide','Total_branching_fraction']).set_index('Radionuclide')

print('There are '+str(len(nonunity))+' radionuclides with a non-unity total for the sum of the branching fractions.')

There are 135 radionuclides with a non-unity total for the sum of the branching fractions.


Sort the DataFrame so that the sum of the branching fractions is increasing.

In [3]:
nonunity.sort_values(by=['Total_branching_fraction'], inplace=True)
nonunity

,Total_branching_fraction
Radionuclide,
At-219,0.970000
U-228,0.975000
Es-254m,0.984410
Es-250,0.985000
Ac-223,0.990000
...,...
Es-255,1.000045
Fr-223,1.000060
Ac-226,1.000060


Considering the radionuclides where the sum of the branching fractions is greater than one first, we see that for all these cases the deviation from unity occurs in the 6<sup>th</sup> significant figure or higher. Therefore these deviations are all consistent with rounding errors given that the branching fractions are recorded in ICRP 107 with up to 5 significant figures of precision.

Cross-checking the radionuclides with the largest sum totals for the branching fractions (Tb-151, Dy-153 etc.) with the original ICRP 107 data file (ICRP-07.NDX), round off errors are clearly the most reasonable explanation. This is because for all these radionuclides there are one or two main decay pathways where the branching fractions add to one, and a minor decay pathway with branching fraction typically of O(10<sup>-5</sup>).

Note the number of radionuclides with total branching fraction greater than one is:

In [4]:
len(nonunity[(nonunity['Total_branching_fraction'] > 1.0)])

81

Now look at the cases where the sum of the branching fractions is less than one and the deviation cannot be assumed to be due to round off errors. The radionuclides for which this applies are:

In [5]:
nonunity[(nonunity['Total_branching_fraction'] < 0.99999)]

,Total_branching_fraction
Radionuclide,
At-219,0.97000
U-228,0.97500
Es-254m,0.98441
Es-250,0.98500
Ac-223,0.99000
Cm-240,0.99700
Po-212m,0.99930
Po-205,0.99940
Np-236,0.99960


Cross-checking the original ICRP 107 data file (ICRP-07.NDX) and another decay data source (<a href="https://www.nndc.bnl.gov/nudat2/chartNuc.jsp">NuDat2</a>), these radionuclides appear to fall into two classes:
1. Round-off of branching fractions in ICRP-07.NDX to the 4<sup>th</sup> or 3<sup>rd</sup> significant figures, which explains the deviation from one (specifically Ag-115, Hf-182m, Np-236, Po-205).
2. Some minor decay pathways not having been included in the ICRP 107 dataset for the other radnionuclides.

The missing minor decay pathways and their approximate branching fractions are:
- Ac-223 electron capture decay (~0.01)
- Am-244m electron capture decay (~0.0004)
- At-217 beta- decay (~0.00012)
- At-219 beta- decay (~0.03)
- Cm-240 electron capture decay (~0.003)
- Es-250 alpha decay (~0.015)
- Es-254m IT decay (~0.01559)
- Po-212m IT decay (~0.0007)
- U-228 electron capture decay (~0.025)

Note that the <a href="https://doi.org/10.11484/jaeri-1347">JAERI 1347</a> and <a href="https://doi.org/10.11484/jaea-data-code-2007-021">JAEA-Data/Code 2007-021</a> reports discuss why these decay pathways were not included in ICRP 107. In short, there were no Evaluated Nuclear Structure Data Files (ENSDF) entries for these decay modes when the ICRP 107 dataset was put together. These reports also confirm that the reason for branching fractions not summing to one for other radionuclides is due to uncertainties in the underlying nuclear data.